# Task 1: Data Ingestion & Preprocessing
This notebook performs Telegram data scraping, preprocessing, and cleaning for Amharic e-commerce posts.


##Install Required Libraries

In [1]:
!pip install telethon pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 16.7 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=3110b4a11760ba09f1c75d0f97474253e95dfa01dca85985d8cee4acb7387560
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


+251 70 428 1724

##Authenticate with Telegram and scrape telegram

In [5]:
from telethon.sync import TelegramClient
from telethon.tl.types import InputPeerChannel
import pandas as pd
import asyncio

api_id = 215----  # Replace with your actual API ID
api_hash = '0657a56fa0d48ee9c3d0--------'  # Replace with your actual API hash
phone = '+251--------'  # Replace with your phone number

async def main():
    client = TelegramClient('session_name', api_id, api_hash)
    await client.start(phone)

    async def scrape_channel(channel_username, limit=100):
        try:
            channel = await client.get_entity(channel_username)
            messages = []
            async for msg in client.iter_messages(channel, limit=limit):
                sender_id = msg.sender.id if msg.sender else None
                messages.append({
                    "date": msg.date,
                    "sender": sender_id,
                    "text": msg.text,
                    "views": msg.views
                })
            return pd.DataFrame(messages)
        except Exception as e:
            print(f"Error scraping {channel_username}: {str(e)}")
            return pd.DataFrame()

    channels = ["@ZemenExpress", "@ethiogebeyam", "@zemengebeya", "https://t.me/HELLOMARKET_AGENT", "https://t.me/efuyegellaMarket", "@abaymart"]  # Remove empty string
    all_data = pd.DataFrame()

    for channel in channels:
        if channel:  # Skip empty strings
            print(f"Scraping {channel}...")
            data = await scrape_channel(channel)
            all_data = pd.concat([all_data, data])

    all_data.to_csv("/content/drive/MyDrive/10Acadamy/data/raw_messages.csv", index=False)
    print("Data saved to '/content/drive/MyDrive/10Acadamy/data/raw_messages.csv'")
    await client.disconnect()

# Run the async main function
if __name__ == '__main__':
    await main()

Scraping @ZemenExpress...
Scraping @ethiogebeyam...
Scraping @zemengebeya...
Scraping https://t.me/HELLOMARKET_AGENT...
Scraping https://t.me/efuyegellaMarket...
Scraping @abaymart...
Data saved to '/content/drive/MyDrive/10Acadamy/data/raw_messages.csv'


##Preprocess Amharic Text
###Clean and normalize text for NER tasks.

In [6]:
import re
import pandas as pd

def clean_amharic_text(text):
  text = re.sub(r'http\S+|@\w+|[^\w\s\u1200-\u137F]', '',str(text))
  text = re.sub(r'\s+', ' ', text).strip()
  return text

df = pd.read_csv('/content/drive/MyDrive/10Acadamy/data/raw_messages.csv')
df['clean_text'] = df['text'].apply(clean_amharic_text)

df.to_csv('/content/drive/MyDrive/10Acadamy/data/cleaned_messages.csv', index=False)
print("Preprocessed data saved to '/content/drive/MyDrive/10Acadamy/data/cleaned_messages.csv'")

Preprocessed data saved to '/content/drive/MyDrive/10Acadamy/data/cleaned_messages.csv'
